# Instalación del AWS SDK para Python

Primero que nada, instalamos boto3, el SDK para Python de AWS

In [ ]:
import os

requisite='boto3'
os.system(f"pip install {requisite}")

Inicio Dynamo

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3, pprint

dynamodb = boto3.resource('dynamodb', endpoint_url="http://172.21.0.5:8000", 
                          aws_access_key_id="<aws_access_key_id>", aws_secret_access_key= "<aws_secret_access_key>", 
                          region_name='us-west-2')


listo las tablas

In [ ]:
response = dynamodb.meta.client.list_tables()

pprint.pprint(response)

creo la tabla de tweets

In [ ]:
table = dynamodb.create_table(
    TableName="Tweets", # Substitute your table name for RetailDatabase
    BillingMode="PROVISIONED",
    KeySchema=[
        {
            'AttributeName': 'id_str',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'timestamp_ms',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'id_str',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'timestamp_ms',
            'AttributeType': 'S'
        }
    ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        },
)

table.meta.client.get_waiter('table_exists').wait(TableName='Tweets')
print('Table created, please continue to insert data.')

instalo el plugin para leer el archivo de jsons separado por nd

In [ ]:
!pip install ndjson

agrego los tweets

In [ ]:
import ndjson, json
from decimal import Decimal

count = 0
ids = set()
with table.batch_writer() as writer:
    with open('../datasets/twitter.ndjson') as f:
        reader = ndjson.reader(f)

        for post in reader:
            if "index" in post:
                pass
            else:
                id = post["id_str"]
                if id in ids:
                    print("duplicate id")
                    pass
                else:
                    ids.add(id)
                    count+=1
                    item = json.loads(json.dumps(post), parse_float=Decimal)
                    writer.put_item(Item=item)
                    if count%100==0:
                        print("Added: {}".format(count))

print(count)

leo los tweets

In [ ]:
count=0
response = table.scan()
print("Items Returned: {}".format(len(response['Items'])))
count+=len(response['Items'])
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    print("Items Returned: {}".format(len(response['Items'])))
    count+=len(response['Items'])

print(count)

borro la tabla

In [ ]:
table.delete()

print(f"Deleting {table.name}...")
table.wait_until_not_exists()

print('Table deleted.')


si en algún momento falla

In [ ]:
dynamodb.meta.client.delete_table(TableName="Tweets")

print('Table deleted.')